In [7]:
import pandas as pd
import requests

# Define station and time range
station = "KMLB"
start_date = "2025-02-01"  # Change to your desired start date
end_date = "2025-02-02"    # Change to your desired end date

# API URL
url = f"https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station={station}&data=all&year1={start_date[:4]}&month1={start_date[5:7]}&day1={start_date[8:]}&year2={end_date[:4]}&month2={end_date[5:7]}&day2={end_date[8:]}&tz=Etc/UTC&format=onlycomma&latlon=yes&direct=yes"

# Fetch the data
response = requests.get(url)
data = response.text

# Save to CSV
file_name = "KMLB_METARtesting.csv"
with open(file_name, "w") as file:
    file.write(data)

# Load into Pandas DataFrame
df = pd.read_csv(file_name)

# Display first rows
df.head()


,station,valid,lon,lat,tmpf,dwpf,relh,drct,sknt,p01i,...,wxcodes,ice_accretion_1hr,ice_accretion_3hr,ice_accretion_6hr,peak_wind_gust,peak_wind_drct,peak_wind_time,feel,metar,snowdepth
0,MLB,2025-02-01 00:00,-80.6453,28.1028,M,M,M,140.00,5.00,M,...,M,M,M,M,M,M,M,M,KMLB 010000Z AUTO 14005KT 10SM CLR 22/18 A3007...,M
1,MLB,2025-02-01 00:05,-80.6453,28.1028,M,M,M,140.00,5.00,M,...,M,M,M,M,M,M,M,M,KMLB 010005Z AUTO 14005KT 10SM CLR 22/18 A3007...,M
2,MLB,2025-02-01 00:10,-80.6453,28.1028,M,M,M,140.00,5.00,M,...,M,M,M,M,M,M,M,M,KMLB 010010Z AUTO 14005KT 10SM CLR 22/18 A3007...,M
3,MLB,2025-02-01 00:15,-80.6453,28.1028,M,M,M,140.00,5.00,M,...,M,M,M,M,M,M,M,M,KMLB 010015Z AUTO 14005KT 10SM CLR 22/18 A3007...,M
4,MLB,2025-02-01 00:20,-80.6453,28.1028,M,M,M,150.00,5.00,M,...,M,M,M,M,M,M,M,M,KMLB 010020Z AUTO 15005KT 10SM CLR 22/18 A3007...,M


In [8]:
import requests
import pandas as pd
import metar

# Define API key and station
api_key = "97624b3d813a41c6baab10280081ed97"
station = "KMLB"

# API URL
url = f"https://api.synopticdata.com/v2/stations/latest?stid={station}&token={api_key}"

# Fetch data
response = requests.get(url)
data = response.json()

# Check if station data exists
if "STATION" in data and len(data["STATION"]) > 0:
    station_data = data["STATION"][0]

    # Extract raw METAR string
    raw_metar = station_data["OBSERVATIONS"].get("metar_value_1", {}).get("value", "N/A")

    # Check if METAR data exists
    if raw_metar != "N/A":
        try:
            # Parse METAR data using the metar package
            metar_obj = metar.Metar(raw_metar)

            # Extract relevant METAR components
            metar_dict = {
                "Station": station_data["STID"],
                "Time": metar_obj.time.isoformat() if metar_obj.time else "N/A",
                "Temperature (°C)": metar_obj.temp.value("C") if metar_obj.temp else "N/A",
                "Dew Point (°C)": metar_obj.dewpt.value("C") if metar_obj.dewpt else "N/A",
                "Wind Speed (kt)": metar_obj.wind_speed.value("KT") if metar_obj.wind_speed else "N/A",
                "Wind Direction (°)": metar_obj.wind_dir.value() if metar_obj.wind_dir else "N/A",
                "Pressure (hPa)": metar_obj.press.value("MB") if metar_obj.press else "N/A",
                "Visibility (miles)": metar_obj.vis.value("MI") if metar_obj.vis else "N/A",
                "Cloud Cover": str(metar_obj.sky) if metar_obj.sky else "N/A",
                "Weather Phenomena": metar_obj.weather if metar_obj.weather else "N/A",
                "Raw METAR": raw_metar
            }

            # Convert dictionary to DataFrame
            df = pd.DataFrame([metar_dict])

            # Replace "M" (missing data) with NaN
            df.replace("M", pd.NA, inplace=True)

            # Save DataFrame to CSV
            csv_filename = "KMLB_METAR_Data.csv"
            df.to_csv(csv_filename, index=False)

            print(f"CSV file saved successfully as: {csv_filename}")
            print(df)
        
        except Exception as e:
            print(f"Error parsing METAR data: {e}")
    else:
        print("No METAR data available.")
else:
    print("No METAR data available.")


Error parsing METAR data: module 'metar' has no attribute 'Metar'


In [9]:
import requests
import pandas as pd
import re

# Define API key and station
api_key = "97624b3d813a41c6baab10280081ed97"
station = "KMLB"

# API URL
url = f"https://api.synopticdata.com/v2/stations/latest?stid={station}&token={api_key}"

# Fetch data
response = requests.get(url)
data = response.json()


def check_station(



# Check if station data exists
if "STATION" in data and len(data["STATION"]) > 0:
    station_data = data["STATION"][0]

    # Extract raw METAR string
    raw_metar = station_data["OBSERVATIONS"].get("metar_value_1", {}).get("value", "N/A")

    # Function to parse METAR manually
    def parse_metar(metar):
        parsed_data = {
            "Station": station_data["STID"],
            "Time": re.search(r"\d{6}Z", metar).group(0) if re.search(r"\d{6}Z", metar) else "N/A",
            "Temperature (°C)": re.search(r" (\d{2})/M?(\d{2}) ", metar),
            "Wind Speed (kt)": re.search(r" (\d{3})(\d{2})KT", metar),
            "Pressure (hPa)": re.search(r" A(\d{4})", metar),
            "Visibility (miles)": re.search(r" (\d{1,2})SM", metar),
            "Cloud Cover": " ".join(re.findall(r" (FEW|SCT|BKN|OVC)\d{3}", metar)),
            "Raw METAR": metar
        }

        # Extract values from regex matches
        if parsed_data["Temperature (°C)"]:
            temp_match = parsed_data["Temperature (°C)"]
            parsed_data["Temperature (°C)"] = int(temp_match.group(1)) if temp_match else "N/A"

        if parsed_data["Wind Speed (kt)"]:
            wind_match = parsed_data["Wind Speed (kt)"]
            parsed_data["Wind Speed (kt)"] = int(wind_match.group(2)) if wind_match else "N/A"

        if parsed_data["Pressure (hPa)"]:
            pressure_match = parsed_data["Pressure (hPa)"]
            parsed_data["Pressure (hPa)"] = int(pressure_match.group(1)) / 100 if pressure_match else "N/A"

        if parsed_data["Visibility (miles)"]:
            visibility_match = parsed_data["Visibility (miles)"]
            parsed_data["Visibility (miles)"] = int(visibility_match.group(1)) if visibility_match else "N/A"

        return parsed_data

    # Check if METAR data exists
    if raw_metar != "N/A":
        metar_data = parse_metar(raw_metar)

        # Convert dictionary to DataFrame
        df = pd.DataFrame([metar_data])

        # Replace "M" (missing data) with NaN
        df.replace("M", pd.NA, inplace=True)

        # Save DataFrame to CSV
        csv_filename = "KMLB_METAR_Data.csv"
        df.to_csv(csv_filename, index=False)

        print(f"CSV file saved successfully as: {csv_filename}")
        print(df)

    else:
        print("No METAR data available.")
else:
    print("No METAR data available.")


CSV file saved successfully as: KMLB_METAR_Data.csv
  Station     Time  Temperature (°C)  Wind Speed (kt)  Pressure (hPa)  \
0    KMLB  191453Z                24                9           30.01   

   Visibility (miles) Cloud Cover  \
0                  10         BKN   

                                           Raw METAR  
0  KMLB 191453Z 21009KT 10SM BKN018 24/20 A3001 R...  


In [6]:
import requests
import pandas as pd
import re

# Define API key and station
api_key = "97624b3d813a41c6baab10280081ed97"
station = "KMLB"

# API URL
url = f"https://api.synopticdata.com/v2/stations/latest?stid={station}&token={api_key}"

# Fetch data
response = requests.get(url)
data = response.json()

# Check if station data exists
if "STATION" in data and len(data["STATION"]) > 0:
    station_data = data["STATION"][0]

    # Extract raw METAR string
    raw_metar = station_data["OBSERVATIONS"].get("metar_value_1", {}).get("value", "N/A")

    # Function to parse METAR manually
    def parse_metar(metar):
        parsed_data = {
            "Station": station_data["STID"],
            "Raw METAR": metar,
        }

        # Extract METAR components using regular expressions
        # Time (6-digit UTC)
        parsed_data["Time"] = re.search(r"\d{6}Z", metar).group(0) if re.search(r"\d{6}Z", metar) else "N/A"

        # Temperature and Dewpoint (e.g., 23/19)
        temp_dew = re.search(r" (\d{2})/M?(\d{2}) ", metar)
        if temp_dew:
            parsed_data["Temperature (°C)"] = int(temp_dew.group(1))
            parsed_data["Dew Point (°C)"] = int(temp_dew.group(2))
        else:
            parsed_data["Temperature (°C)"] = "N/A"
            parsed_data["Dew Point (°C)"] = "N/A"

        # Wind (e.g., 18012KT)
        wind = re.search(r" (\d{3})(\d{2})KT", metar)
        if wind:
            parsed_data["Wind Direction (°)"] = int(wind.group(1))
            parsed_data["Wind Speed (kt)"] = int(wind.group(2))
        else:
            parsed_data["Wind Direction (°)"] = "N/A"
            parsed_data["Wind Speed (kt)"] = "N/A"

        # Pressure (e.g., A2997 => 1010 hPa)
        pressure = re.search(r" A(\d{4})", metar)
        if pressure:
            parsed_data["Pressure (hPa)"] = int(pressure.group(1)) / 100
        else:
            parsed_data["Pressure (hPa)"] = "N/A"

        # Visibility (e.g., 10SM)
        visibility = re.search(r" (\d{1,2})SM", metar)
        if visibility:
            parsed_data["Visibility (miles)"] = int(visibility.group(1))
        else:
            parsed_data["Visibility (miles)"] = "N/A"

        # Cloud cover (e.g., BKN020)
        clouds = re.findall(r" (FEW|SCT|BKN|OVC)(\d{3})", metar)
        if clouds:
            parsed_data["Cloud Cover"] = ", ".join([f"{cloud[0]} {cloud[1]}" for cloud in clouds])
        else:
            parsed_data["Cloud Cover"] = "N/A"

        return parsed_data

    # Check if METAR data exists
    if raw_metar != "N/A":
        metar_data = parse_metar(raw_metar)

        # Convert dictionary to DataFrame
        df = pd.DataFrame([metar_data])

        # Replace "M" (missing data) with NaN
        df.replace("M", pd.NA, inplace=True)

        # Save DataFrame to CSV
        csv_filename = "KMLB_METAR_Data.csv"
        df.to_csv(csv_filename, index=False)

        print(f"CSV file saved successfully as: {csv_filename}")
        print(df)

    else:
        print("No METAR data available.")
else:
    print("No METAR data available.")


CSV file saved successfully as: KMLB_METAR_Data.csv
  Station                                          Raw METAR     Time  \
0    KMLB  METAR KMLB 191445Z AUTO 21008KT 10SM SCT018 24...  191445Z   

   Temperature (°C)  Dew Point (°C)  Wind Direction (°)  Wind Speed (kt)  \
0                24              20                 210                8   

   Pressure (hPa)  Visibility (miles) Cloud Cover  
0           30.01                  10     SCT 018  
